This code is from download_products_table.py which can be refacted to work for this too. 

In [14]:
from yaml import safe_load as yaml_load
from sqlalchemy import create_engine
import pandas as pd
from os.path import isfile
images_table_location = "data/images_table.json"

if isfile(images_table_location) == False:
    with open(".gitignore/credentials_for_marketplace.yml") as file:
        credentials  = yaml_load(file)
    DATABASE_TYPE = credentials['DATABASE_TYPE']
    DBAPI = credentials['DBAPI'] 
    ENDPOINT = credentials['ENDPOINT']       
    USER = credentials['DBUSER']
    PASSWORD = credentials['DBPASSWORD']
    PORT = credentials['PORT']
    DATABASE = credentials['DATABASE']

    print("Connecting to database...")
    engine = create_engine(f"{DATABASE_TYPE}+{DBAPI}://{USER}:{PASSWORD}@{ENDPOINT}:{PORT}/{DATABASE}")        
    df = pd.read_sql_table("images", engine)     

    print("Saved as ", images_table_location )
    df.to_json(images_table_location)

In [15]:
import pandas as pd
product_df = pd.read_json("data/products_table_clean.json")
image_df = pd.read_json("data/images_table.json")

In [16]:
image_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 12604 entries, 0 to 12603
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   id           12604 non-null  object        
 1   product_id   12604 non-null  object        
 2   bucket_link  12604 non-null  object        
 3   image_ref    12604 non-null  object        
 4   create_time  12604 non-null  datetime64[ns]
dtypes: datetime64[ns](1), object(4)
memory usage: 590.8+ KB


Now I will define a function that takes in an image file name and retrieves the category.

In [17]:
def get_image_category(image_name):
    
    image_row = image_df.loc[image_df["id"] == image_name]
    product_id = image_row.iloc[0]["product_id"]
    product_row = product_df.loc[product_df["id"] == product_id]
    category = product_row.iloc[0]["main_category"]
    
    return category


In [18]:
get_image_category("912bb259-3ad9-457b-9db1-ce1da9016057")

'home_garden'

The issue with this particular section of code is that the length of the list of images is higher than the list of images in the image table, indicating that there are some files in the folder which aren't recorded in the database.

In [19]:
# import glob
# import numpy as np

# images_folder = "data/cleaned_images_128"
# jpg_locations = images_folder + "/*.jpg"
# list_of_images = glob.glob(jpg_locations)

# X_file_locations = np.zeros(len(list_of_images))
# y = np.zeros(len(list_of_images))

# for image_location in list_of_images:
#     print(image_location)
#     image_name = image_location.split("/")[-1].removesuffix(".jpg")
#     image_category = get_image_category(image_name)
    


In [20]:
images = set(image_df["id"])
images_folder = "data/cleaned_images_64"

In [21]:
from torchvision.transforms import ToTensor
import torch
from PIL import Image

def process_image(image_location):
    image = Image.open(image_location)
    image = ToTensor()(image)
    image = torch.flatten(image)
    return image.numpy()
    

In [22]:
def convert_to_image_array(image):
    image_category = get_image_category(image)
    image_location = images_folder +"/"+ image +".jpg"
    image_array = process_image(image_location)
    return image_array, image_category
     

In [23]:
def create_dict_of_categories(df, column_to_index): 
    categories = set(df[column_to_index]) 
    categories_dict = {k: v for v, k in enumerate(categories)} 
    return categories_dict

category_numbers = create_dict_of_categories(product_df, "main_category")

In [24]:
import numpy as np 
from alive_progress import alive_bar

array_size = 64*64*3

#TODO this bit is messy so fix it
images = list(images)

# n = 20000
n = len(images)
X = np.zeros((n,array_size))
y = np.zeros(n)



with alive_bar(n) as bar:
    for index in range(n):
        print("image number :", index)
        image = images[index]
        try:
            features, label = convert_to_image_array(image)
            X[index, :] = features
            y[index] = category_numbers[label]

        except:
            #TODO deal with this because it might be affecting the model
            X[index, :] = np.zeros(array_size)
            y[index] = 0

        bar()

on 0: image number : 0
on 1: image number : 1
on 2: image number : 2
on 3: image number : 3
on 4: image number : 4
on 5: image number : 5
on 6: image number : 6
on 7: image number : 7
on 8: image number : 8
on 9: image number : 9
on 10: image number : 10
on 11: image number : 11
on 12: image number : 12
on 13: image number : 13
on 14: image number : 14
on 15: image number : 15
on 16: image number : 16
on 17: image number : 17
on 18: image number : 18
on 19: image number : 19
on 20: image number : 20
on 21: image number : 21
on 22: image number : 22
on 23: image number : 23
on 24: image number : 24
on 25: image number : 25
on 26: image number : 26
on 27: image number : 27
on 28: image number : 28
on 29: image number : 29
on 30: image number : 30
on 31: image number : 31
on 32: image number : 32
on 33: image number : 33
on 34: image number : 34
on 35: image number : 35
on 36: image number : 36
on 37: image number : 37
on 38: image number : 38
on 39: image number : 39
on 40: image number 

In [ ]:

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)

In [ ]:

from sklearn.linear_model import LogisticRegression
 
model = LogisticRegression(max_iter=100)
model.fit(X_train, y_train)

ValueError: This solver needs samples of at least 2 classes in the data, but the data contains only one class: 0.0

In [ ]:

score = model.score(X_test, y_test)
print(score)

0.12166666666666667
